In [1]:
import sims_pars.prob as dist

In [3]:
di = 'gamma(0.1, rate = 1)'

%timeit dist.parse_distribution(di)

%timeit dist.complete_function(di)
seq = dist.complete_function(di)
print(seq)
%timeit dist.parse_distribution(seq, to_complete=False)

471 µs ± 5.48 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
32 µs ± 589 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
gamma(shape=0.1, rate=1)
411 µs ± 2.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
from sims_pars import bayes_net_from_script, sample

In [5]:
bn = bayes_net_from_script('''
PCore Root {
    x ~ binom(prob = 0.5, size = 5)
    y ~ unif(0.01, 0.02)
}
''')

In [6]:
def fn1():
    bayes_net_from_script('''
PCore Root {
    x ~ binom(prob = 0.5, size = 5)
    y ~ unif(0.01, 0.02)
}
''')


def fn():
    bn2 = bayes_net_from_script('''
    PCore Root {
        x = 60
        t ~ binom(prob = 0.3, size = x)
        u ~ unif(0.01, 0.02)
    }
    ''')


    bn.extend(bn2)

%timeit fn1()
%timeit fn()

247 µs ± 3.74 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
412 µs ± 4.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
import logging

In [9]:
log = logging.getLogger('Test')

In [12]:
log.addHandler(logging.StreamHandler())

In [14]:
log.warning("info")



info


In [17]:
log.hasHandlers()

True